In [ ]:
import os
import sys
parent_dir = os.path.dirname(os.getcwd())
sys.path.append(parent_dir)
utils_dir = os.path.join(parent_dir, "src", "utils")
sys.path.append(utils_dir)
import pandas as pd
import plotly.graph_objects as go
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from process_data import process_data
from weighted_accuracy import weighted_accuracy_scorer
from plot_learning_curves import plot_learning_curves

In [ ]:
X_path= os.path.join("..", "data","input", "X_train_Wwou3IE.csv")
X_preprocessed = pd.read_csv(X_path, delimiter=',')
y_path= os.path.join("..", "data","input", "y_train_jJtXgMX.csv")
y_preprocessed = pd.read_csv(y_path, delimiter=',')

In [ ]:
cold_rate = {
    1: 100,  # January
    2: 90,   # February
    3: 70,   # March
    4: 50,   # April
    5: 30,   # May
    6: 10,   # June
    7: 0,    # July
    8: 5,    # August
    9: 20,   # September
    10: 40,  # October
    11: 60,  # November
    12: 80   # December
}

X_preprocessed['DELIVERY_START'] = pd.to_datetime(X_preprocessed['DELIVERY_START'], utc = True)

X_preprocessed['month'] = X_preprocessed['DELIVERY_START'].dt.month
X_preprocessed['cold_rate'] = X_preprocessed['month'].map(cold_rate)
X_preprocessed.drop('month', axis=1, inplace=True)

In [ ]:
X = process_data(X_preprocessed.copy(deep=True), "predicted_spot_price", None, "standard")
y = process_data(y_preprocessed.copy(deep=True), None, None, None)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

# Initialize the KNN model
# You can adjust 'n_neighbors' based on your validation tests to find the best value
knn = KNeighborsClassifier(n_neighbors=5)

# Fit the model on your training data
knn.fit(X_train, y_train)

# Predict on your test data
predictions = knn.predict(X_test)

# If you have the actual test labels, you can compute the accuracy as follows:
# y_test = pd.read_csv('path_to_y_test.csv').squeeze() # Assuming you have actual labels for test data
# accuracy = accuracy_score(y_test, predictions)
# print(f"Accuracy: {accuracy}")

# If you don't have actual labels for test set, use predictions as your submission
submission = pd.DataFrame(predictions, columns=['Predicted_Spot_ID_Delta'])
submission.to_csv('submission.csv', index=False)
